In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
start = '2015-01-01'
overall = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
symbols = overall.Symbol.to_list()
print(symbols)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DV

In [29]:
removed = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[1][['Date', 'Removed']]
removed = removed.set_index(removed.Date.Date)
removed.index = pd.to_datetime(removed.index)
removed = removed[removed.index >= start]
removed = removed.Removed.dropna()
removed

,Ticker,Security
Date,,
2024-12-23,QRVO,Qorvo
2024-12-23,AMTM,Amentum
2024-12-23,CTLT,Catalent
2024-11-26,MRO,Marathon Oil
2024-10-01,BBWI,"Bath & Body Works, Inc."
...,...,...
2015-03-23,AVP,Avon Products
2015-03-18,CFN,Carefusion
2015-03-12,PETM,PetSmart


In [30]:
symbols.extend(removed.Ticker.to_list())

def save_to_csv(dataframe, file_path="data.csv"):
    dataframe.to_csv(file_path, index=True)

def load_from_csv(file_path="data.csv"):
    try:
        return pd.read_csv(file_path, index_col=0, parse_dates=True)
    except pd.errors.EmptyDataError:
        # Handle any unexpected empty data errors
        return pd.DataFrame()

def fetch_missing_data(symbols, file_path="data.csv"):
    existing_data = load_from_csv(file_path)
    existing_symbols = existing_data.columns if not existing_data.empty else []
    missing_symbols = [symbol for symbol in symbols if symbol not in existing_symbols]

    if missing_symbols:
        print(f"Fetching data for: {missing_symbols}")
        new_data = yf.download(missing_symbols, start=start)['Close']

        # Drop columns where all rows are NaN
        valid_data = new_data.dropna(axis=1, how="all")

        if valid_data.empty:
            print("No valid data fetched. Not updating the file.")
            return existing_data

        print(f"Fetched valid data for: {valid_data.columns.tolist()}")
        updated_data = pd.concat([existing_data, valid_data], axis=1)
        save_to_csv(updated_data, file_path)
        return updated_data

    print("No missing data to fetch.")
    return existing_data


df = fetch_missing_data(symbols)

df


Fetching data for: ['BRK.B', 'BF.B', 'PXD', 'ATVI', 'DISH', 'FRC', 'SIVB', 'ABMD', 'FBHS', 'TWTR', 'NLSN', 'CTXS', 'DRE', 'CERN', 'PBCT', 'XLNX', 'GPS', 'KSU', 'MXIM', 'ALXN', 'HFC', 'FLIR', 'VAR', 'CXO', 'TIF', 'NBL', 'ETFC', 'ADS', 'AGN', 'RTN', 'ARNC', 'XEC', 'WCG', 'VIAB', 'CELG', 'TSS', 'APC', 'RHT', 'LLL', 'DWDP', 'XL', 'GGP', 'DPS', 'MON', 'WYN', 'CHK', 'BCR', 'LVLT', 'SPLS', 'WFM', 'BBBY', 'MNK', 'RAI', 'YHOO', 'MJN', 'SWN', 'FTR', 'LLTC', 'ENDP', 'STJ', 'LM', 'DO', 'TYC', 'CPGX', 'TE', 'CVC', 'BXLT', 'ARG', 'TWC', 'SNDK', 'CAM', 'ESV', 'GMCR', 'BRCM', 'CMCSK', 'SIAL', 'HCBK', 'JOY', 'HSP', 'DTV', 'FDO', 'KRFT', 'QEP', 'LO', 'WIN', 'AGN', 'DNR', 'AVP', 'CFN', 'SWY', 'PXD', 'ATVI', 'DISH', 'FRC', 'SIVB', 'ABMD', 'FBHS', 'TWTR', 'NLSN', 'CTXS', 'DRE', 'CERN', 'PBCT', 'XLNX', 'GPS', 'KSU', 'MXIM', 'ALXN', 'HFC', 'FLIR', 'VAR', 'CXO', 'TIF', 'NBL', 'ETFC', 'ADS', 'AGN', 'RTN', 'ARNC', 'XEC', 'WCG', 'VIAB', 'CELG', 'TSS', 'APC', 'RHT', 'LLL', 'DWDP', 'XL', 'GGP', 'DPS', 'MON', 'WYN'

[*********************100%***********************]  89 of 89 completed

89 Failed downloads:
['NLSN', 'TWTR', 'NBL', 'LM', 'FBHS', 'TIF', 'ALXN', 'ETFC', 'MXIM', 'AVP', 'ESV', 'ARNC', 'FLIR', 'ENDP', 'PBCT', 'SIVB', 'DRE', 'CXO', 'DNR', 'XLNX', 'XEC', 'LLL', 'APC', 'PXD', 'MON', 'ATVI', 'CELG', 'RTN', 'MNK', 'XL', 'SWN', 'HFC', 'FTR', 'WIN', 'ABMD', 'VIAB', 'CTXS', 'YHOO', 'CERN', 'VAR', 'DTV', 'DO', 'DISH', 'RHT', 'GPS', 'DWDP', 'ADS', 'QEP', 'TSS', 'BRK.B', 'AGN', 'FRC', 'BBBY', 'KSU', 'WCG', 'CHK']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['HCBK', 'ARG', 'CAM', 'MJN', 'KRFT', 'CMCSK', 'SNDK', 'GMCR', 'CPGX', 'SPLS', 'CFN', 'HSP', 'BF.B', 'BCR', 'BRCM', 'FDO', 'SIAL', 'DPS', 'LVLT', 'TYC', 'TWC', 'TE', 'SWY', 'JOY', 'BXLT', 'LO', 'WFM', 'LLTC', 'GGP', 'RAI', 'CVC', 'STJ', 'WYN']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-02)')


No valid data fetched. Not updating the file.


,A,AA,AAL,AAP,AAPL,ABBV,ACE,ACGL,ACN,ADBE,...,UHS,VFC,VMC,VNO,VNT,WHR,WMT,YUM,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.353008,36.027794,51.079910,142.627319,24.347178,43.563183,NaN,19.496668,75.312714,72.339996,...,106.474159,51.941383,61.653030,57.969090,NaN,137.164902,23.441494,43.035698,100.930573,40.278938
2015-01-05,36.653107,33.940552,51.051491,140.747284,23.661278,42.743366,NaN,19.379999,74.041130,71.980003,...,104.296097,51.504784,59.950169,58.114803,NaN,134.336502,23.373266,42.161304,104.695595,40.037128
2015-01-06,36.082127,34.190105,50.255581,140.648361,23.663502,42.531796,NaN,19.423332,73.507050,70.529999,...,104.334488,51.201977,59.812099,58.585861,NaN,133.227844,23.553377,41.643806,103.808128,39.646530
2015-01-07,36.561008,35.074921,50.227154,143.670746,23.995312,44.250778,NaN,19.536667,75.049919,71.110001,...,107.452782,51.856880,60.944283,59.824219,NaN,136.361877,24.178303,43.023808,106.389862,40.464928
2015-01-08,37.656910,36.073174,50.843040,144.930084,24.917269,44.713585,NaN,19.876667,76.194374,72.919998,...,108.556236,53.025829,62.849655,60.028164,NaN,139.879654,24.688616,43.773285,107.519379,41.088036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,135.848907,38.610001,17.350000,43.730000,258.200012,180.000000,NaN,92.669998,361.630005,447.940002,...,181.220001,21.879999,264.309998,42.419998,37.110001,116.550003,92.680000,135.000000,107.059036,164.699997
2024-12-26,135.579407,38.349998,17.350000,44.240002,259.019989,179.199997,NaN,92.930000,360.429993,450.160004,...,181.130005,22.120001,264.549988,42.650002,37.349998,116.029999,92.790001,136.000000,107.009148,165.520004
2024-12-27,135.289932,37.680000,17.350000,44.060001,255.589996,178.009995,NaN,92.339996,356.179993,446.480011,...,180.460007,21.770000,262.359985,41.790001,36.610001,115.169998,91.660004,135.229996,106.630005,164.600006


In [55]:
df.index = pd.to_datetime(df.index)

print(removed[removed.Ticker == 'SBNY'])

print(df.shape[1])
df = df.dropna(axis=1, how='all')
print(df.shape[1])


def pricefilter_remove(ticker):
    df[ticker] = df[ticker][df[ticker].index <= removed[removed.Ticker == ticker].index[0]]

def pricefilter_add(ticker):
    # print(overall[overall.Symbol == ticker])
    # print(overall[overall.Symbol == ticker].values[0])
    date_added = overall.loc[overall.Symbol == ticker, 'Date added'].iloc[0]
    df[ticker] = df[ticker][df[ticker].index >= date_added]

for ticker_rem in removed.Ticker:
    if ticker_rem in df.columns:
        pricefilter_remove(ticker_rem)

for ticker_add in overall.Symbol:
    if ticker_add in df.columns:
        pricefilter_add(ticker_add)

df[['LUMN']].head(20)

           Ticker        Security
Date                             
2023-03-15   SBNY  Signature Bank
598
598


,LUMN
Date,
2015-01-02,19.831503
2015-01-05,19.410721
2015-01-06,19.170282
2015-01-07,19.270472
2015-01-08,19.465826
2015-01-09,19.235403
2015-01-12,19.360632
2015-01-13,19.335583
2015-01-14,19.190321
